<a href="https://colab.research.google.com/github/kluo9/HM-personalized-fashion-recommendation/blob/main/hm_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of ranking stage is to give rankings of the items recalled based on features from item and customer. 

The training is achieved with input as features from item and customers, and out = 1/0 based on whether this item is purchased ot not.

This becomes a classification problem, and it is very imbalanced since the items with output label 1 is very small compared to 0. So I generate the negative samples (out label = 0) so that the negative sample size is about 30 * the positive (out label = 1) sample size.

I set the training data set to be train4, which are transactions made between 2020-08-15 to 2020-08-23. I will use the customers during this period to train the ranking model. To be specific, each transaction in train4 will be a sample: the user embedding is then generated from train1 to train3. 


In [64]:
import pandas as pd
import numpy as np
import pickle
import datetime
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_GT_vali", "rb") as fp: 
     user_vali_GT = pickle.load(fp)

In [4]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_recall_vali", "rb") as fp: 
     user_vali_recall = pickle.load(fp)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [6]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
transaction_df = pd.read_csv('/content/transactions_train.csv.zip')

 97% 568M/584M [00:05<00:00, 106MB/s]
100% 584M/584M [00:05<00:00, 114MB/s]


In [8]:
transaction_df["t_dat"] = pd.to_datetime(transaction_df["t_dat"])
train1 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,9,8)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,16))]
train2 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,9,1)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,8))]
train3 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,8,23)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,1))]
train4 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,8,15)) & (transaction_df['t_dat'] < datetime.datetime(2020,8,23))]

val = transaction_df.loc[transaction_df["t_dat"] >= datetime.datetime(2020,9,16)]

In [9]:
del transaction_df

# Featue engineering

I will split the train data into train_ref and train4, and use train_ref to generate features for train4 customers, and use train4 to train ranking model

In [10]:
train_ref = pd.concat([train1, train2, train3], axis=0)

Possible features for ranking:

Features from customers:
1. Customer embedding (remove those customers with no purchasing history)
2. Customer age
3. Customer club_member_status
4. Customer fashion_news_frequency

Features from items
1. Item embedding
2. Item product_group
3. Item color
4. Item section_name

## Customer features

Customer embedding for the training set 

In [11]:
# extract customers in train4:
train4_customer = train4['customer_id']

In [12]:
# read item embedding
with open("/content/gdrive/MyDrive/hm_recommendation/item_embedding", "rb") as fp: 
     item_embedding = pickle.load(fp)

For each customer in train4_customer, calculate customer embedding based on their previous purchases (in train_ref).

In [53]:
train_ref['article_id'] = train_ref['article_id'].astype("string")
items_per_user = train_ref.groupby(['customer_id'])['article_id'].apply(list) 

In [35]:
def customer_embedding(purchase_history, customer, item_vectors):
  customer_embed = [0] * 10
  if customer in purchase_history.keys():
    items = purchase_history[customer]
    for item in items:
      customer_embed = np.add(customer_embed, item_vectors[item])
    customer_embed = np.divide(customer_embed, len(items))
  return customer_embed


In [57]:
embedding_all_customer = []
for user in train4_customer:
  embedding_all_customer.append(customer_embedding(items_per_user, user, item_embedding))

Customer age

In [65]:
! kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
customers_df = pd.read_csv('/content/customers.csv.zip')
customers_df["age"].fillna(int(stats.mode(customers_df["age"])[0]), inplace = True)

customers.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [66]:
user_age = []
for user in train4_customer:
  user_age.append(customers_df['age'][customers_df['customer_id']==user])

KeyboardInterrupt: ignored

In [ ]:
user_age